# GPU Simulator


## GPU Qiskit Aer Simulator Backends and Methods

 Following Qiskit Aer backends currently support GPU acceleration:
* `QasmSimulator`
* `StatevectorSimulator`
* `UnitarySimulator`

To check the availability of GPU support on these backends, `available_method()` returns methods with gpu suports.

## Introduction

This notebook shows how to accelerate Qiskit Aer simulators by using GPUs. 

To install GPU support in Qiskit Aer, please install GPU version of Qiskit Aer by

`pip install qiskit-aer-gpu`


### Note 

Qiskit Aer only supports NVIDIA's GPUs and requires CUDA toolkit installed on the system. 

In [2]:
from qiskit import *
from qiskit.circuit.library import *
from qiskit.providers.aer import *
qasm_sim = QasmSimulator()
print(qasm_sim.available_methods())

['automatic', 'statevector', 'statevector_thrust', 'density_matrix', 'density_matrix_thrust', 'stabilizer', 'matrix_product_state', 'extended_stabilizer']


If Qiskit Aer with GPU support is installed correctly, you can see `statevector_gpu` and `density_matrix_gpu`

In [3]:
st_sim = StatevectorSimulator()
print(st_sim.available_methods())
u_sim = UnitarySimulator()
print(u_sim.available_methods())

['automatic', 'statevector', 'statevector_thrust']
['automatic', 'unitary', 'unitary_thrust']


### Simulation with GPU

Here is a simple example to run quantum volume circuit with 20 qubits by using `QasmSimulator` backend.
Setting the simulation method `statevector_gpu` in `backend_options` parameter passed to `QasmSimulator.run` method to use GPU for the simulaiton.

In [7]:
shots = 64
qubit = 20
depth=10
qv20 = QuantumVolume(qubit, depth, seed = 0)
qv20 = transpile(qv20, backend=qasm_sim, optimization_level=0)
qv20.measure_all()
qobj = assemble(qv20, shots=shots, memory=True)
if 'statevector_gpu' in qasm_sim.available_methods():
    result = qasm_sim.run(qobj, backend_options={"method" : "statevector_gpu"}).result()
else:
    print('There is no GPU simulator installed, so using CPU simulator')
    result = qasm_sim.run(qobj, backend_options={"method" : "statevector"}).result()

counts = result.get_counts(qv20)
print(counts)

There is no GPU simulator installed, so using CPU simulator
{'00101000010110000101': 1, '10010110001010110100': 1, '01001001000010000001': 1, '01111011101010111101': 1, '00101001101111000001': 1, '00111001000111000010': 1, '01100001001011000001': 1, '11000111010100100010': 1, '10110001001011000101': 1, '01000101000010000100': 1, '11001111011001100101': 1, '10000011010110111111': 1, '01011001101001001101': 1, '00100000010011100110': 1, '10111001111101101001': 1, '11110011001101000100': 1, '11101110001000111101': 1, '10010110011111000110': 1, '00010000111110110101': 1, '11101101111001000010': 1, '01111101111101110100': 1, '00011001001101000111': 1, '01110111110100101010': 1, '10111011000011101010': 1, '00111001111000101110': 1, '01111000100001010011': 1, '11111010100111000000': 1, '00000011100100000110': 1, '00000100100001101001': 1, '00100011111011000101': 1, '00001100000000111010': 1, '10110010101111001011': 1, '00100001101010001110': 1, '00001110101100010111': 1, '00110000010101110010

The following sample shows an example using `density_matrix_gpu` mthod in `QasmSimulator`.

In [8]:
qubit = 10
depth = 10
qv10 = QuantumVolume(qubit, depth, seed = 0)
qv10 = transpile(qv10, backend=qasm_sim, optimization_level=0)
qv10.measure_all()
qobj = assemble(qv10, shots=shots, memory=True)
if 'density_matrix_gpu' in qasm_sim.available_methods():
    result = qasm_sim.run(qobj, backend_options={"method" : "density_matrix_gpu"}).result()
else:
    print('There is no GPU simulator installed, so using CPU simulator')
    result = qasm_sim.run(qobj, backend_options={"method" : "density_matrix"}).result()

counts = result.get_counts(qv10)
print(counts)

There is no GPU simulator installed, so using CPU simulator
{'0100100100': 1, '1111000000': 1, '1010011100': 1, '0010101010': 1, '1101100011': 1, '0000110110': 1, '1101010000': 1, '1010110100': 1, '0010000101': 1, '1001000010': 1, '0110000100': 1, '1000111101': 1, '1111101110': 1, '1001000110': 1, '0000010011': 1, '0000100110': 1, '0101101000': 1, '0110111111': 1, '0110001110': 1, '1111001000': 1, '0010000100': 1, '0101011101': 1, '0100001111': 1, '0101001000': 3, '0010010010': 1, '1110100000': 1, '1110001110': 1, '0100110100': 1, '1101101010': 1, '0001101110': 1, '0110101011': 1, '1100001101': 1, '1100011100': 1, '0001000011': 1, '1000111011': 1, '1000000001': 1, '0011111000': 1, '0010010000': 2, '0000101000': 1, '0110001010': 1, '1111101101': 1, '0010111011': 3, '1101110101': 1, '1001000000': 1, '1111010111': 1, '1110011010': 1, '1101110010': 1, '0000000110': 1, '0101111010': 1, '0001111100': 1, '0000101001': 1, '0110100001': 1, '1101100110': 1, '0110101110': 1, '0000011111': 1, '111

## Parallelizing Simulaiton by Using Multiple GPUs

In general GPU has less memory size than CPU, and the largest number of qubits is depending on the memory size. For example, if a GPU has 16 GB of memory, Qiskit Aer can simulate up to 29 qubits by using `statevector_gpu` method in `QasmSimulator` and `StatevectorSimulator` backends or up to 14 qubits by using `density_matrix_gpu` method in `QasmSimulator` backend and `unitary_gpu` method in `UnitarySimulator` backend in double precision.

To simulate with more larger nnumber of qubits, multiple GPUs can be used to parallelize the simulation or also parallel simulation can accelerate the simulation speed. 

To use multi-GPUs, following options should be set in the `backend_options` parameter passed to `run` method. In the parallel simulator, the vector of quantum states are divided into sub-vectors called chunk and chunks are distributed to memory of multiple-GPUs. 

Following 2 options should be passed:
* `blocking_enable` : Set `True` to enable parallelization
* `blocking_qubits` : This option sets the size of chunk that is distributed to parallel memory space. Set this parameter to satisfy `16*(2^(blocking_qubits+4)) < smallest memory size on the system (in byte)` for double precision. (`8*` for single precision).

The parameter `blocking_qubits` will be varied in different environment, so this parameter is optimized by using some benchmarks before running actual applications. Usually setting 20 to 23 will be good for many environments. 

Here is an example of Quantum Volume of 30 qubits with multiple GPUs by using `QasmSimulator` backend and `statevector_gpu` method.

In [ ]:
qubit = 30
depth = 10
qv30 = QuantumVolume(qubit, depth, seed = 0)
qv30 = transpile(qv30, backend=qasm_sim, optimization_level=0)
qv30.measure_all()
qobj = assemble(qv30, shots=shots, memory=True)
if 'statevector_gpu' in qasm_sim.available_methods():
    result = qasm_sim.run(qobj, backend_options={"method" : "statevector_gpu", "blocking_enable" : True, "blocking_qubits" : 23 }).result()
    counts = result.get_counts(qv30)
    print(counts)
else:
    print('There is no GPU simulator installed')

### Note

Note that only `QasmSimulator` can be applied for large qubit circuits because `StatevectorSimulator` and `UnitarySimulator` backends currently returns snapshots of state that will require large memory space. If CPU has enough memory to store snapshots these 2 backends can be used with GPUs.

## Distribution of Shots by Using Multiple GPUs

Also GPUs can be used to accelerate simulating multiple shots with noise models. If the system has multiple GPUs, shots are automatically distributed to GPUs if there is enough memory to simulate one shot on single GPU. Also if there is only one GPU on the system, multiple shots can be parallelized on threads of GPU.

Note multiple shots distribution on GPU is slower than running on CPU when number of qubits to be simulated is small because of large overheads of GPU kernel launch. 

Following example shows running 1000 shots of quantum volume circuit with noise on GPU.

In [10]:
from qiskit.providers.aer.noise import *
noise_model = NoiseModel()
error = depolarizing_error(0.05, 1)
noise_model.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3'])
shots = 1000
qobj = assemble(qv10, shots=shots, memory=True)
if 'statevector_gpu' in qasm_sim.available_methods():
    result = qasm_sim.run(qobj, noise_model = noise_model, backend_options={"method" : "statevector_gpu"}).result()
else:
    print('There is no GPU simulator installed, so using CPU simulator')
    result = qasm_sim.run(qobj, noise_model = noise_model, backend_options={"method" : "statevector"}).result()

rdict = result.to_dict()
print("simulation time = {0}".format(rdict['time_taken']))

There is no GPU simulator installed, so using CPU simulator
simulation time = 0.1777939796447754


In [12]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright